In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leer una tabla completa
df = pd.read_sql("SELECT * FROM grado;", engine)
df.head()

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropología Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
2,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
4,5,Artes Escénicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...


In [3]:
df.shape

(229, 5)

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': 'fbf8ded8fbf0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jGnwEyW4SfulV9yVxiFUYg', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)

In [7]:
# indice semántico

index_mapping = {
    "properties": {
        "id": { "type": "integer" },        
        "nombre": { "type": "text" },     
        "id_area": { "type": "integer" },
        "descripcion": { "type": "text" },
        "descripcion_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        },
        "salidas": { "type": "text" },
        "salidas_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

index_name = "informacion_grados"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(
    index=index_name,
    mappings=index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'informacion_grados'})

In [8]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())


/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import json
from elasticsearch.helpers import bulk

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


In [23]:
import json

# Query 1

res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 190, Nombre Título: Ingeniería en Organización Industrial, Description: Un Ingeniero de Organización/Ingeniero en Organización Industrial es un profesional ca paz de analizar y resolver problemas y opti mizar los procesos de una empresa hacién dola más eficaz y eficiente. Puede identificar y ejecutar las prioridades estratégicas de la compañía, los indicadores clave, los recursos disponibles, los beneficios y costes espera dos, las limitaciones tecnológicas y la respon sabilidad social corporativa. El Ingeniero de Organización es el profesional idóneo para las empresas del sector industrial y de servicios que requieren directivos con una amplia formación técnica. Su formación dual, como técnico y como gestor, le permite tener una visión global de la empresa y su entorno., Score: 4.1358886

Document ID: 192, Nombre Título: Ingeniería en Electrónica y Automática Industrial, Description: Estas titulaciónes tienen un campo de estudio centrado en todo lo que es 

In [15]:
# Query
res = es.search(
    index=index_name,
    size=3,
    query={
        "match": {
            "salidas": "profesor"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 48, Nombre Título: Lenguas Modernas y sus Literaturas, Description: La existencia en la UCM de las filologías mo dernas cuenta ya con más de medio siglo y tienen una sólida reputación nacional e inter nacional, como demuestran los numerosos convenios con las principales universidades nacionales y extranjeras, que han dado lugar a la creación de numerosas redes de investi gación. Las transformaciones sociales deriva das de la constitución de un espacio europeo, los nuevos planteamientos de las relaciones internacionales, la presencia emergente de ciudadanos que proceden de otras culturas y el desarrollo de las tecnologías, plantean la necesidad de una titulación basada en una formación de calidad, desde esta obligada óptica de relación intercultural, a partir del necesario conocimiento de varias lenguas y sus respectivas culturas y literaturas., Score: 5.012066

Document ID: 29, Nombre Título: Estudios Semíticos e IslámicosÁrea de Estudios Semíticos e Islámi

In [14]:
# Query semantica

query_sentence = "análisis de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")
    

Search Results:
Document ID: 84, Nombre Título: Análisis de Negocios, Description: Este grado se centra en el análisis de datos masivos en el ámbito de los negocios que permitan establecer relaciones significativas. Es una carrera bilingüe de alto contenido tecnológico formada por tres grandes pilares: . Administración y Dirección de Empresas, de análisis estadístico y de informática, con materias tales como Management Analytics, Data Mining, Business Intelligence, Big Data, Matemáticas, Contabilidad y Auditoría. . Derecho, Fiscalidad, Economía, Humani dades en la Era Digital. . Informática aplicada al Business Analytics y Business English. Algunas de las competencias básicas y gene rales a adquirir por el alumno: Organización, sistematización y planificación en la identificación de problemas, pautas y modelos en el contexto del big data. Análisis de datos a gran escala procedentes de fuentes audiovisuales, textos y numéricas. Capacidad para el cumplimiento de obje tivos, resolución de